In [ ]:
import panel as pn
import param

from aiida import load_profile
from aiida.orm import load_node
from utils_visualizer import get_id_from_user
load_profile()

pn.extension()

class DetailView(param.Parameterized):
    
    # Get the uuid/pk from user
    wc_id = param.String(default=get_id_from_user(), doc='WorkChain id')

    units = 'eV'

    @param.depends('wc_id')
    def show_node(self): 
        from utils_visualizer import node_info_string
        wcnode = load_node(self.wc_id)
        column = pn.Column(pn.pane.Markdown(node_info_string(wcnode)))
        return column
    
    @param.depends('wc_id','units')
    def show_output_parameters(self):
        from utils_visualizer import out_info_string
        out_dict = load_node(self.wc_id).outputs.output_parameters.get_dict()
        column = pn.Column(pn.pane.Markdown(out_info_string(out_dict, self.units)))
        return column
        
    @param.depends('wc_id')
    def show_structure(self):
        from utils_visualizer import structure_jsmol
        
        cifdata = load_node(self.wc_id).outputs.output_structure.get_cif()
        cif_str = cifdata.get_content()
        
        column = pn.Column(pn.pane.Bokeh(structure_jsmol(cif_str=cif_str)))
        return column    
    
    @param.depends('wc_id')
    def show_last_input_parameters(self):
        from utils_visualizer import dict_to_string
        import yaml
        param_dict = load_node(self.wc_id).outputs.last_input_parameters.get_dict()
        column = pn.Column(pn.pane.Markdown("## CP2K input parameters (last)"))
        column.append(pn.pane.Markdown(dict_to_string(param_dict)))
        return column
         
    @param.depends('wc_id','units')
    def show_steps(self):
        from utils_visualizer import plot_steps
        
        out_dict = load_node(self.wc_id).outputs.output_parameters.get_dict()

        column = pn.Column(pn.pane.Bokeh(plot_steps(out_dict, self.units)))
        return column
      
explorer = DetailView()

gspec = pn.GridSpec(sizing_mode='stretch_both', max_width=900)
gspec[0, 0] = explorer.param
gspec[1, 0] = explorer.show_node
gspec[2, 0] = explorer.show_output_parameters
gspec[3, 0] = explorer.show_structure
gspec[4, 0] = explorer.show_steps
gspec[5, 0] = explorer.show_last_input_parameters
gspec.servable()
